In [ ]:
!pip install -q transformers datasets sentence-transformers hnswlib accelerate bitsandbytes peft


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.8 MB/s 

In [ ]:

# Step 2: Import libraries
import pandas as pd
import numpy as np
import hnswlib
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sentence_transformers import SentenceTransformer
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline,
    BitsAndBytesConfig,
)
from peft import get_peft_model, LoraConfig, TaskType
from google.colab import files, drive

# Step 3: Mount Drive
drive.mount('/content/drive')

# Step 4: Upload CSV
uploaded = files.upload()  # Upload ARS_DATA.csv
df = pd.read_csv("ARS_DATA.csv")

# Step 5: Clean and encode data
def clean_and_encode_data(df):
    df.dropna(inplace=True)
    label_cols = ['Gender', 'Dietary Preferences', 'Fitness Goal', 'Exercise Level', 'Meal Name', 'Meal Time']
    encoders = {col: LabelEncoder() for col in label_cols}
    for col in label_cols:
        df[col + '_original'] = df[col]
        df[col] = encoders[col].fit_transform(df[col])
    scaler = StandardScaler()
    df[['Height (cm)', 'Weight (kg)', 'BMI']] = scaler.fit_transform(df[['Height (cm)', 'Weight (kg)', 'BMI']])
    return df, encoders, scaler

df, encoders, scaler = clean_and_encode_data(df)

# Step 6: Create profile description
df['Full_Description'] = df.apply(lambda row:
    f"{row['Gender_original']}, {row['Age']} years old, {row['Height (cm)']:.2f} cm, {row['Weight (kg)']:.2f} kg, "
    f"{row['Dietary Preferences_original']} diet, {row['Fitness Goal_original']}, "
    f"{row['Exercise Level_original']} exercise, BMI {row['BMI']:.2f}",
    axis=1)

# Step 7: Generate embeddings
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
embeddings = embedder.encode(df['Full_Description'].tolist(), convert_to_numpy=True)

# Step 8: Build HNSW index
dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.init_index(max_elements=len(embeddings), ef_construction=100, M=16)
index.add_items(embeddings, ids=list(range(len(embeddings))))
index.set_ef(100)

# Step 9: Prepare data for fine-tuning
df['input'] = df['Full_Description'].apply(lambda x: f"{x} => Recommended meal and workout:")
df['output'] = df['Meal Name_original'].astype(str)
df['text'] = df['input'] + " " + df['output']
ds = Dataset.from_pandas(df[['text']])

# Step 10: Load base model & LoRA
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
quant_config = BitsAndBytesConfig(load_in_8bit=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(base_model, lora_config)
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Step 11: Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = ds.map(tokenize_function, batched=True)
small_ds = tokenized_dataset.select(range(min(7000, len(tokenized_dataset))))

# Step 12: Train
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/facebook/opt-350m",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir='/content/drive/MyDrive/facebook_opt-350m_logs',
    report_to="none",
    fp16=True
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()

# Step 13: Save artifacts
model.save_pretrained("/content/drive/MyDrive/RAGfacebook_opt-350m")
tokenizer.save_pretrained("/content/drive/MyDrive/RAGfacebook_opt-350mk")
df.to_csv("/content/drive/MyDrive/ARS_dfs.csv", index=False)
np.save("/content/drive/MyDrive/ARSs_embeddings.npy", embeddings)
index.save_index("/content/drive/MyDrive/ARSs_hnsw_index.bin")

# Step 14: Load for inference
df = pd.read_csv("/content/drive/MyDrive/ARS_dfs.csv")
embeddings = np.load("/content/drive/MyDrive/ARSs_embeddings.npy")
dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.load_index("/content/drive/MyDrive/ARSs_hnsw_index.bin")
index.set_ef(100)
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/RAGfacebook_opt-350mk")
base_model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
from peft import PeftModel
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/RAGfacebook_opt-350m")
model.eval()
gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Step 15: RAG inference
def recommend_with_rag(query, embedder, index, df, gen_pipeline, top_k=5):
    query_vec = embedder.encode([query])
    ids, _ = index.knn_query(query_vec, k=top_k)
    retrieved_contexts = df.iloc[ids[0]][['Full_Description', 'Meal Name_original', 'Meal Time']].values.tolist()
    context_blocks = []
    for desc, meal, time in retrieved_contexts:
        context_blocks.append(f"{desc} => Meal: {meal} at {time}")
    rag_context = "\n".join(context_blocks)
    prompt = f"""You are a personalized nutrition and fitness recommender.

Based on past users with similar profiles:

{rag_context}

Now, for the following user:
{query}
Provide a recommended meal and workout plan:"""
    response = gen_pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.9)[0]['generated_text']
    return response

# Step 16: Test query
query = "Female, 25 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5"
print(recommend_with_rag(query, embedder, index, df, gen_pipeline))


Mounted at /content/drive


Saving ARS_DATA.csv to ARS_DATA.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

<ipython-input-2-f74d17e64227>:104: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'Cohere

You are a personalized nutrition and fitness recommender.

Based on past users with similar profiles:

Female, 60 years old, -1.56 cm, 1.16 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.22 => Meal: Quinoa Bowl at 2
Female, 40 years old, 1.23 cm, 1.42 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 0.25 => Meal: Quinoa Bowl at 1
Female, 48 years old, -1.35 cm, 1.21 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.05 => Meal: Tofu Salad at 0
Female, 27 years old, -1.63 cm, 0.26 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 1.35 => Meal: Grilled Chicken at 0
Female, 22 years old, 1.50 cm, -0.79 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI -1.27 => Meal: Beef Stir Fry at 3

Now, for the following user:
Female, 25 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5
Provide a recommended meal and workout plan: Keto Salad, Protein Smoothie, Protein Smoothie: Paneer Stir Fry, Protein Smoothie: Tofu Salad, Protein Smoo

In [ ]:
!pip install -q transformers sentence-transformers hnswlib peft

import numpy as np
import pandas as pd
import hnswlib
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
from google.colab import drive
drive.mount('/content/drive')
# Load CSV, embeddings, HNSW index
df = pd.read_csv("/content/drive/MyDrive/ARS_dfs.csv")
embeddings = np.load("/content/drive/MyDrive/ARSs_embeddings.npy")

# Load HNSW index
dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.load_index("/content/drive/MyDrive/ARSs_hnsw_index.bin")
index.set_ef(100)

# Load embedder
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# Load LoRA model + tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/RAGfacebook_opt-350mk")
base_model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/RAGfacebook_opt-350m")
model.eval()

# Text generation pipeline
gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def recommend_with_rag(query, embedder, index, df, gen_pipeline, top_k=3):
    query_vec = embedder.encode([query])
    ids, _ = index.knn_query(query_vec, k=top_k)
    retrieved_contexts = df.iloc[ids[0]][['Full_Description', 'Meal Name_original', 'Meal Time']].values.tolist()
    context_blocks = [f"{desc} => Meal: {meal} at {time}" for desc, meal, time in retrieved_contexts]
    rag_context = "\n".join(context_blocks)

    prompt = f"""You are a personalized nutrition and fitness recommender.

Based on past users with similar profiles:

{rag_context}

Now, for the following user:
{query}
Provide a recommended meal and workout plan:"""

    response = gen_pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.9)[0]['generated_text']
    return response


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

In [ ]:
query = "Female, 25 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5"
print(recommend_with_rag(query, embedder, index, df, gen_pipeline))


You are a personalized nutrition and fitness recommender.

Based on past users with similar profiles:

Female, 60 years old, -1.56 cm, 1.16 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.22 => Meal: Quinoa Bowl at 2
Female, 40 years old, 1.23 cm, 1.42 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 0.25 => Meal: Quinoa Bowl at 1
Female, 48 years old, -1.35 cm, 1.21 kg, Keto diet, Muscle Gain, High (5x/week) exercise, BMI 2.05 => Meal: Tofu Salad at 0

Now, for the following user:
Female, 25 years old, 170 cm, 65 kg, keto diet, muscle gain, high exercise, BMI 22.5
Provide a recommended meal and workout plan: Veggie Wrap, Quinoa Bowl, Protein Smoothie, Protein Smoothie, High (5x/week) exercise, BMI 2.45 => Meal: Chickpea Curry, Protein Smoothie, Protein Smoothie, High (5x/week) exercise, BMI 2.38 => Meal: Tofu Salad, Quinoa Bowl, Protein Smoothie, High (5x/week) exercise, BMI 2.40 => Meal: Protein Smoothie, Bowl, Protein Smoothie,
